<a href="https://colab.research.google.com/github/Divyesh-Kanagavel/deep_learning--keras/blob/master/GPT_tokenizer_Andrej_Karpathy_lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tokenization : a very important part of nlp applications. an independent pipeline which has its own training needs, bytecode implementations etc


the goal is to get to a state where there are no tokenizations required and the transformer just works on byte streams of data. There is already a paper published ,but not yet proved at scale.

the unicode standard is used to encode text in python and many other programmign languanges. in python, for a given character, ord function is used to retrieve the unicode.

In [4]:
ord("h")

104

Even emojis like hi , laughing emojis are given unicode integer code.

In [5]:
ord("😂")

128514

In [6]:
("👋🏻 , hi as a emoji").encode("utf-8")

b'\xf0\x9f\x91\x8b\xf0\x9f\x8f\xbb , hi as a emoji'

with encode("utf-8") method, the output is a sequence of non-readable bytes shown in hex format. if it is read as list, it gives a neat sequence of integers.


In [7]:
# prompt: l

l = list(("👋🏻 , hi as a emoji").encode("utf-8"))
print(l)


[240, 159, 145, 139, 240, 159, 143, 187, 32, 44, 32, 104, 105, 32, 97, 115, 32, 97, 32, 101, 109, 111, 106, 105]


In [8]:
l = list(("👋🏻 , hi as a emoji").encode("utf-16"))
print(l)

[255, 254, 61, 216, 75, 220, 60, 216, 251, 223, 32, 0, 44, 0, 32, 0, 104, 0, 105, 0, 32, 0, 97, 0, 115, 0, 32, 0, 97, 0, 32, 0, 101, 0, 109, 0, 111, 0, 106, 0, 105, 0]


we see that in utf-16 , there are zeros in between, meaning that a lot of bits are wasted compared to utf-8

if we use UTF-8 raw byte streams as inputs to transformers, it could be the ultimate goal, but it has a problem, for given raw texts, the raw byte encodings will be very long and the transformers will not be able to attend to a very large past or future data.
so, algos like bytepair encoding is used. here, the sequence is scanned and if there are particular sub sequences of strings which occur often, they are replaced by another character which is then appended to the vocab.
For example:
aabcaabdaabc
Z = aa
ZbcZbdZbc
now Y=bc
ZYZbdZY


In [9]:
text = "You can quickly replace text with emoji suggestions as you type.Enter a commonly used word or phrase like “heart” or “thumbs up”, then press Fn-E or the Globe key-E.If there are no emoji suggestions for the text you entered, the Character Viewer opens.Press Return to replace the text with the suggested emoji, or choose another suggestion.Click the Arrow button  to see additional suggestions and emoji in the Character Viewer.Tip: You can automatically insert an emoji every time you type certain text (for example, automatically insert 😆 when you type XD). See Replace text and punctuation in documents."
tokens = text.encode('utf-8')
tokens = list(map(int,tokens)) # convert to a list of integers
print("the length of text is: ", len(text))
print("the length of tokens is : ", len(tokens))
print("the list of tokens : ", tokens)


the length of text is:  606
the length of tokens is :  617
the list of tokens :  [89, 111, 117, 32, 99, 97, 110, 32, 113, 117, 105, 99, 107, 108, 121, 32, 114, 101, 112, 108, 97, 99, 101, 32, 116, 101, 120, 116, 32, 119, 105, 116, 104, 32, 101, 109, 111, 106, 105, 32, 115, 117, 103, 103, 101, 115, 116, 105, 111, 110, 115, 32, 97, 115, 32, 121, 111, 117, 32, 116, 121, 112, 101, 46, 69, 110, 116, 101, 114, 32, 97, 32, 99, 111, 109, 109, 111, 110, 108, 121, 32, 117, 115, 101, 100, 32, 119, 111, 114, 100, 32, 111, 114, 32, 112, 104, 114, 97, 115, 101, 32, 108, 105, 107, 101, 32, 226, 128, 156, 104, 101, 97, 114, 116, 226, 128, 157, 32, 111, 114, 32, 226, 128, 156, 116, 104, 117, 109, 98, 115, 32, 117, 112, 226, 128, 157, 44, 32, 116, 104, 101, 110, 32, 112, 114, 101, 115, 115, 32, 70, 110, 45, 69, 32, 111, 114, 32, 116, 104, 101, 32, 71, 108, 111, 98, 101, 32, 107, 101, 121, 45, 69, 46, 73, 102, 32, 116, 104, 101, 114, 101, 32, 97, 114, 101, 32, 110, 111, 32, 101, 109, 111, 106, 105, 32, 1

the difference between length of tokens and length of text is because some characters more than one byte.

In [10]:
#dictionary to calculate pair wise occurences of characters
def get_stats(ids):
    counts = {}
    for pair in zip(ids,ids[1:]):
        counts[pair] = counts.get(pair,0)+1
    return counts

In [11]:
counts = get_stats(tokens)

In [12]:
print(sorted(((v,k) for (k,v) in counts.items()), reverse=True))

[(21, (101, 32)), (20, (32, 116)), (13, (116, 104)), (12, (104, 101)), (12, (101, 114)), (11, (114, 32)), (11, (110, 32)), (10, (116, 101)), (10, (32, 97)), (9, (116, 105)), (8, (111, 110)), (8, (32, 101)), (7, (116, 32)), (7, (115, 32)), (7, (114, 101)), (7, (111, 114)), (7, (101, 115)), (6, (117, 32)), (6, (115, 101)), (6, (111, 117)), (6, (110, 115)), (6, (109, 111)), (6, (105, 111)), (6, (101, 120)), (6, (97, 110)), (6, (32, 115)), (5, (121, 32)), (5, (120, 116)), (5, (117, 103)), (5, (116, 111)), (5, (115, 117)), (5, (115, 116)), (5, (111, 106)), (5, (106, 105)), (5, (105, 110)), (5, (103, 103)), (5, (103, 101)), (5, (101, 110)), (5, (101, 109)), (5, (100, 32)), (5, (97, 99)), (5, (32, 111)), (5, (32, 99)), (4, (226, 128)), (4, (121, 111)), (4, (114, 116)), (4, (112, 108)), (4, (112, 101)), (4, (108, 121)), (4, (105, 99)), (4, (105, 32)), (4, (99, 101)), (4, (99, 97)), (4, (97, 114)), (4, (44, 32)), (4, (32, 121)), (4, (32, 119)), (4, (32, 105)), (3, (121, 112)), (3, (117, 116)), 

(101,32) is the pair which has the highest number of occurences.


In [13]:
chr(101), chr(32)

('e', ' ')

e and space are the most commonly occuring pair of tokens

In [14]:
top_pair = max(counts, key=counts.get)

In [15]:
top_pair

(101, 32)

Once we have the number of occurences, we can start to merge the pair and produce a new token number - so vocab size increase and the sequence length decreases - there is usually a sweet spot which is a hyperparameter and is tunable

In [16]:
def merge(ids, pair, idx):
  new_ids = [] # new list of indices
  i = 0
  while(i < len(ids)):
    if (i < len(ids) - 1) and ids[i] == pair[0] and ids[i+1] == pair[1]:
      new_ids.append(idx)
      i+=2
    else:
      new_ids.append(ids[i])
      i+=1
  return new_ids

new_ids = merge([5,6,6,7,9,1], [6,7], 99)
print(new_ids)

[5, 6, 99, 9, 1]


In [17]:
# prompt: merge our tokens with top_pair as the second argument and replace it with 256

new_tokens = merge(tokens, top_pair, 256)
print(new_tokens)
print("length of new tokens is : ",len(new_tokens))


[89, 111, 117, 32, 99, 97, 110, 32, 113, 117, 105, 99, 107, 108, 121, 32, 114, 101, 112, 108, 97, 99, 256, 116, 101, 120, 116, 32, 119, 105, 116, 104, 32, 101, 109, 111, 106, 105, 32, 115, 117, 103, 103, 101, 115, 116, 105, 111, 110, 115, 32, 97, 115, 32, 121, 111, 117, 32, 116, 121, 112, 101, 46, 69, 110, 116, 101, 114, 32, 97, 32, 99, 111, 109, 109, 111, 110, 108, 121, 32, 117, 115, 101, 100, 32, 119, 111, 114, 100, 32, 111, 114, 32, 112, 104, 114, 97, 115, 256, 108, 105, 107, 256, 226, 128, 156, 104, 101, 97, 114, 116, 226, 128, 157, 32, 111, 114, 32, 226, 128, 156, 116, 104, 117, 109, 98, 115, 32, 117, 112, 226, 128, 157, 44, 32, 116, 104, 101, 110, 32, 112, 114, 101, 115, 115, 32, 70, 110, 45, 69, 32, 111, 114, 32, 116, 104, 256, 71, 108, 111, 98, 256, 107, 101, 121, 45, 69, 46, 73, 102, 32, 116, 104, 101, 114, 256, 97, 114, 256, 110, 111, 32, 101, 109, 111, 106, 105, 32, 115, 117, 103, 103, 101, 115, 116, 105, 111, 110, 115, 32, 102, 111, 114, 32, 116, 104, 256, 116, 101, 120, 11

In [18]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids,ids[1:]):
        counts[pair] = counts.get(pair,0)+1
    return counts

def merge(ids, pair, idx):
  new_ids = [] # new list of indices
  i = 0
  while(i < len(ids)):
    if (i < len(ids) - 1) and ids[i] == pair[0] and ids[i+1] == pair[1]:
      new_ids.append(idx)
      i+=2
    else:
      new_ids.append(ids[i])
      i+=1
  return new_ids

vocab_size = 276 # desired vocab_size
num_merges = vocab_size-256
ids = list(tokens) # copy of tokens
merges = {}
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats,key=stats.get)
  idx = 256+i
  ids = merge(ids,pair,idx)
  print(f'merging {pair} into {idx}')
  merges[pair] = idx


merging (101, 32) into 256
merging (32, 116) into 257
merging (101, 114) into 258
merging (110, 32) into 259
merging (116, 105) into 260
merging (257, 104) into 261
merging (116, 32) into 262
merging (32, 101) into 263
merging (101, 115) into 264
merging (115, 32) into 265
merging (111, 114) into 266
merging (111, 117) into 267
merging (109, 111) into 268
merging (260, 111) into 269
merging (261, 256) into 270
merging (97, 99) into 271
merging (116, 101) into 272
merging (120, 262) into 273
merging (116, 104) into 274
merging (268, 106) into 275


In [19]:
print("Original list length: ", len(tokens))
print("Merged list length : ", len(ids))
print("Compression ratio : ", len(tokens)/ len(ids))

Original list length:  617
Merged list length :  461
Compression ratio :  1.3383947939262473


let us do the same for a bigger text.

In [20]:
bigger_text = ''' I’m assuming, if you are on the Internet and reading kind of a nerdy blog, that you know what Unicode is. At the very least, you have a very general understanding of it — maybe “it’s what gives us emoji”.

That’s about as far as most people’s understanding extends, in my experience, even among programmers. And that’s a tragedy, because Unicode has a lot of… ah, depth to it. Not to say that Unicode is a terrible disaster — more that human language is a terrible disaster, and anything with the lofty goals of representing all of it is going to have some wrinkles.

So here is a collection of curiosities I’ve encountered in dealing with Unicode that you generally only find out about through experience. Enjoy.

Also, I strongly recommend you install the Symbola font, which contains basic glyphs for a vast number of characters. They may not be pretty, but they’re better than seeing the infamous Unicode lego.

Some definitions

There are already plenty of introductions to Unicode floating around (wikipedia, nedbat, joel), and this is not going to be one. But here’s a quick refresher.

Unicode is a big table that assigns numbers (codepoints) to a wide variety of characters you might want to use to write text. We often say “Unicode” when we mean “not ASCII”, but that’s silly since of course all of ASCII is also included in Unicode.

UTF-8 is an encoding, a way of turning a sequence of codepoints into bytes. All Unicode codepoints can be encoded in UTF-8. ASCII is also an encoding, but only supports 128 characters, mostly English letters and punctuation.

A character is a fairly fuzzy concept. Letters and numbers and punctuation are characters. But so are Braille and frogs and halves of flags. Basically a thing in the Unicode table somewhere.

A glyph is a visual representation of some symbol, provided by a font. It might represent a single character, or it might represent several. Or both!

Unicode is divided into seventeen planes, numbered zero through sixteen. Plane 0 is also called the Basic Multilingual Plane, or just BMP, so called because it contains the alphabets of most modern languages. The other planes are much less common and are sometimes informally referred to as the astral planes.

Everything you know about text is wrong

If the only written languge you’re familiar with is English, that goes doubly so.

Perhaps you want to sort text. A common enough problem. Let’s give this a try in Python. To simplify things, we’ll even stick to English text.

1
2
3
4
>>> words = ['cafeteria', 'caffeine', 'café']
>>> words.sort()
>>> words
['cafeteria', 'caffeine', 'café']
Oops. Turns out Python’s sorting just compares by Unicode codepoint, so the English letter “é” (U+00E9) is greater than the English letter “f” (U+0066).

Did you know the German letter “ß” is supposed to sort equivalently to “ss”? Where do you sort the Icelandic letter “æ”? What about the English ligature “æ”, which is the same character?

What about case? The Turkish dotless “ı” capitalizes to the familiar capital “I”, but in Turkish, the lowercase of that is “ı” and the uppercase of “i” is “İ”. Is uppercase “ß” the more traditional “SS”, or maybe “Ss”, or the somewhat recent addition “ẞ”?

Or, how do you compare equality? Is “ß” equal to “ss”? Is “æ” equal to “ae”? Is “é” equal to “é”?

Ah, you say! I’ve heard about this problem and know how to solve it. I can just throw Unicode normalization at it, which will take care of combining characters and all that other nonsense. I can even strip out all combining characters and have nice normal English text left, because for some reason I am under the impression that English text is “normal” and all else is “abnormal”.

Sure, let’s give that a try.

1
2
3
4
5
>>> import unicodedata
>>> normalize = lambda s: ''.join(ch for ch in unicodedata.normalize('NFKD', s) if not unicodedata.combining(ch))
>>>
>>> normalize('Pokémon')
'Pokemon'
Great, problem solved.

1
2
3
4
5
6
>>> normalize('ı')
'ı'
>>> normalize('æ')
'æ'
>>> normalize('ß')
'ß'
Hmm.

1
2
3
4
>>> normalize('한글')
'한글'
>>> normalize('イーブイ')
'イーフイ'
Uh oh.

Yes, it turns out that Unicode decomposition also decomposes Hangul (the alphabet used to write Korean) into its sub-components, which then may or may not still even render correctly, as well as splitting the diacritics off of Japanese kana, which significantly alters the pronunciation and meaning. Almost as if Unicode decomposition was never meant to help programmers forcibly cram the entire world back into ASCII.

Even if you only care about English text, there’s more than one Latin alphabet in Unicode! Is “x” equivalent to “𝗑” or “𝘅” or “𝘹” or “𝙭” or “𝚡” or “ｘ” or “𝐱”? What about “×” or “х” or “⨯” or “ⅹ”? Ah, sorry, those last four are actually the multiplication sign, a Cyrillic letter, the symbol for cross product, and the Roman numberal for ten.

This is a particularly aggravating problem because most programming languages have facilities for comparing and changing the case of text built in, and most of them are extremely naïve about it. You can’t even correctly change the case of English-looking text without knowing what locale it came from — the title-case of “istanbul” may actually be “İstanbul” depending on language, because of Turkish’s dotted “i”.

The only library I’m aware of off the top of my head for correctly dealing with any of these problems is ICU, which is a hulking monstrosity hardly suited for shipping as part of a programming language. And while their homepage does list a lot of impressive users, I’ve only encountered it in code I’ve worked on once.

Combining characters and character width

Typically we think of combining characters as being the floating diacritical marks that can latch onto the preceding letter, such as using U+0301 COMBINING ACUTE ACCENT to make “q́”, in case we are direly in need of it for some reason. There are a few other combining “diacriticals” that aren’t so related to language; for example, U+20E0 COMBINING ENCLOSING CIRCLE BACKSLASH can produce “é⃠”, the universal symbol for “my software only supports English, and also I am not aware that English has diacritics too”. Or perhaps you’d use U+20E3 COMBINING ENCLOSING KEYCAP to make “é⃣” and indicate that the user should press their é key.

All of these have an impact on the “length” of a string. You could write either of those “é” sequences with three codepoints: the letter “e”, the combining accent, and the combining border. But clearly they each only contribute one symbol to the final text. This isn’t a particularly difficult problem; just ignore combining characters when counting, right?

More interesting are the Unicode characters that are not combining characters, but compose in some way in practice anyway. The flag emoji, for example, don’t actually exist in Unicode. The Unicode Consortium didn’t want to be constantly amending a list of national flags as countries popped in and out of existence, so instead they cheated. They added a set of 26 regional indicator symbols, one for each letter of the English alphabet, and to encode a country’s flag you write its two-letter ISO country code with those symbols. So the Canadian flag, 🇨🇦, is actually the two characters U+1F1E8 REGIONAL INDICATOR SYMBOL LETTER C and U+1F1E6 REGIONAL INDICATOR SYMBOL LETTER A. But if you put a bogus combination together, you probably won’t get a flag glyph; you’ll get stand-ins for the characters instead. (For example, 🇿🇿.) So the “length” of a pair of these characters depends both on the display font (which may not support all flags), and on the current geopolitical state of the world. How’s that for depending on global mutable state?

But it gets better! There’s a character called U+200D ZERO WIDTH JOINER, which is used to combine otherwise distinct characters in some languages (but has fairly general semantics). Apple has made creative use of this character to compose emoji together. The report on emoji has some examples. So now the length of some text is completely arbitrary, based on whatever arbitrary ligatures the font includes.

To be fair, that was already true anyway. You might argue that the length of text in human terms is not actually all that interesting a quantity, and you’d be right, but that’s why this section is about character width. Because I’m typing in a terminal right now, and terminals fit all their text in a grid.

Let’s return to the simpler world of letters and revisit that Hangul example:

1
2
>>> normalize('한글')
'한글'
Hangul characters are actually blocks composed of exactly three parts called Jamo. (Here’s gritty detail on Hangul, Jamo, and Unicode. It’s a really cool alphabet.) Applying Unicode decomposition actually breaks each character down into its component Jamo, which are then supposed to render exactly the same as the original. They aren’t marked as combining characters in the Unicode database, but if you have three of them in a row (arranged sensibly), you should only see one character. The actual decomposition for the text above is “ㅎㅏㄴ ㄱㅡㄹ”, written with separate characters that don’t combine. There are a good few languages that work this way — Devanagari (the script used for Hindi et al.) and Bengali rely heavily on character composition, and Hebrew uses it for rendering vowels.

And yet I ended up with four very different renderings. In this blog post, with my default monospace font, I see the full sequence of six Jamo. If I paste the same text somewhere with a proportional font, I see something very nearly identical to the original characters, albeit slightly fuzzier from being generated on the fly. In Konsole, I see only the first Jamo for each character: 'ㅎㄱ'. (This has been fixed as of July 6, 2016, though I don’t know what Konsole release contains the fix.) And in my usual libvte-based terminal, the combining behavior falls apart, and I see a nonsensical mess that I can’t even reproduce with Unicode:

Screenshot of mangled Hangul in a terminal; several characters overlap

I can only guess at what happened here. Clearly both terminals decided that each set of three Jamo was only one character wide, but for some reason they didn’t combine. Konsole adamantly refuses to render any Jamo beyond the first, even if I enter them independently; VTE dutifully renders them all but tries to constrain them to the grid, leading to overlap.

This is not the first width-related problem I’ve encountered with Unicode and terminals. Consider emoji, which tend to be square in shape. I might reasonably want to say to someone on IRC: “happy birthday! 🎁 hope it’s a good one.” (That’s U+1F380 WRAPPED PRESENT, if you didn’t take my advice and install Symbola.) But I use a terminal IRC client, and here’s how that displays, in VTE and Konsole:

Screenshot of the sentence in VTE; the birthday gift overlaps the following space

Screenshot of the sentence in Konsole; the spacing is correct but the cursor position is wrong

You can see how VTE has done the same thing as with Hangul: it thinks the emoji should only take up one character cell, but dutifully renders the entire thing, allowing the contents to spill out and overlap the following space. You might think Konsole has gotten this one right, but look carefully — the final quote is slightly overlapping the cursor. Turns out that Konsole will print each line of text as regular text, so any character that doesn’t fit the terminal grid will misalign every single character after it. The cursor (and selection) is always fit to the grid, so if you have several emoji in the same row, the cursor might appear to be many characters away from its correct position. There are several bugs open on Konsole about this, dating back many years, with no response from developers. I actually had to stop using Konsole because of this sole issue, because I use ⚘ U+2698 FLOWER as my shell prompt, which misaligned the cursor every time.

All of these problems can be traced back to the same source: a POSIX function called wcwidth, which is intended to return the number of terminal columns needed to display a given character. It exists in glibc, which sent me on a bit of a wild goose chase. I originally thought that wcwidth must be reporting that the second and third Jamo characters are zero width, but this proved not to be the case:

1
2
3
4
>>> libc.wcwidth(c_wchar('\u1100'))  # initial Jamo
2
>>> libc.wcwidth(c_wchar('\u1161'))  # second Jamo
1
Well, it seems Konsole actually implements its own wcwidth which appears to be based on this original implementation. Both versions preserve an innocuous comment that explains quite a lot:

Hangul Jamo medial vowels and final consonants (U+1160-U+11FF) have a column width of 0.

Aha. So Konsole saw that the second and third Jamo took zero space, so it didn’t bother trying to print them at all.

Then what the hell is VTE doing? It defers to some utility functions in glib (GNOME’s library of… stuff), such as g_unichar_iszerowidth, which… explicitly says yes for everything between U+1160 and U+1200. Wouldn’t you know it, those are the secondary and tertiary Jamo characters. So VTE saw that they took zero space, so it didn’t make any extra room for them, but still tried to print them. I expect they didn’t combine in VTE because VTE has no idea they’re supposed to combine, so it printed each one individually.

Oh, but this madness gets even better. WeeChat, another terminal IRC client, outright strips emoji, everywhere. This is apparently the fault of… glibc’s implementation of wcwidth, which defaults to 1 for printable characters and 0 otherwise, which requires knowing what the characters are, which oops doesn’t work so well when glibc was using a vendored copy of the (pre-emoji) Unicode 5.0 database until glibc 2.22, which was released less than a month ago.

Beloved SSH replacement mosh has a similar problem, in this case blamed on the wcwidth implementation shipped with OS X. Gosh, I thought Apple was on the ball with Unicode.

We’re now up to at least four mutually incompatible and differently broken versions of this same function. Lovely.

I might be on the fringe here, but I’m pretty adamant that having a communication program silently and invisibly eat parts of your text is a bad thing.

While I’m at it: why are emoji left with a width of 1? They tend to be drawn to fit a square, just like CJK characters (which are why we need double-width character support in the first place), and they’re even of Japanese origin. My rendering problems would go away in both terminals if they used widths of 2. Hell, I’m going to go file bugs on both of them right now.

You will not go to space today

Sometimes you care about whitespace. Perhaps you’re using it to separate words. In, say, a programming language. Like JavaScript.

1
alert(2+ 40);
JavaScript’s syntax defers the decision of what counts as whitespace to the Unicode database, which assigns a WSpace property to a handful of codepoints. Seems like a good approach, except for this one unusual exception: ” ” is a space character, U+1680 OGHAM SPACE MARK. Ogham is an alphabet used in older forms of Irish, and its space character generally renders as a line. Surprise!

Complicating this somewhat further, there are actually two definitions of whitespace in Unicode. Unicode assigns every codepoint a category, and has three categories for what sounds like whitespace: “Separator, space”; “Separator, line”; and “Separator, paragraph”.

If you’re familiar with Unicode categories, you might be tempted to use these to determine what characters are whitespace. Except that CR, LF, tab, and even vertical tab are all categorized as “Other, control” and not as separators. You might think that at least LF should count as a line separator, but no; the only character in the “Separator, line” category is U+2028 LINE SEPARATOR, and the only character in “Separator, paragraph” is U+2029 PARAGRAPH SEPARATOR. I have never seen either of them used, ever. Thankfully, all of these have the WSpace property.

As an added wrinkle, the lone oddball character “⠀” renders like a space in most fonts. But it’s not whitespace, it’s not categorized as a separator, and it doesn’t have WSpace. It’s actually U+2800 BRAILLE PATTERN BLANK, the Braille character with none of the dots raised. (I say “most fonts” because I’ve occasionally seen it rendered as a 2×4 grid of open circles.) Everything is a lie.

JavaScript has no string type

JavaScript’s “String” type (or “string” type?) is not actually a string type. Observe:

1
2
var bomb = "💣";
console.log(bomb.length);  // 2
That’s a string containing a single character, U+1F4A3 BOMB. Yet JavaScript thinks it contains two! What on earth is going on here? Let’s see what JavaScript thinks those two characters are, using charCodeAt.

1
2
console.log(bomb.charCodeAt(0).toString(16));  // d83d
console.log(bomb.charCodeAt(1).toString(16));  // dca3
These aren’t actually characters. Everything from U+D800 through U+DFFF is permanently reserved as a non-character for the sake of encoding astral plane characters in UTF-16. The short version is that all BMP characters are two bytes in UTF-16, and all astral plane characters are two of these non-characters (called a surrogate pair) for a total of four bytes.

JavaScript’s string type is backed by a sequence of unsigned 16-bit integers, so it can’t hold any codepoint higher than U+FFFF and instead splits them into surrogate pairs. I argue that a string isn’t a string if it can’t hold a sequence of arbitrary characters, and JavaScript strings can’t directly contain astral plane characters, so they don’t qualify.

I rag on JavaScript, but this is an old problem. C strings (well, char*) are just sequences of bytes, so you can’t fit more than Latin-1. Some libraries have historically tried to address this with “wide strings”, wchar_t*, but the size of wchar_t is implementation-defined and 16 bits on Windows, where the entire OS API has the same problem as JavaScript.

Arguably, 16-bit faux strings are worse than 8-bit faux strings. It becomes pretty obvious pretty quickly that 8 bits is not enough to fit more than some European alphabets, and anyone but the most sheltered programmer is forced to deal with it the first time they encounter an em dash. But 16 bits covers the entire BMP, which contains all current languages, some ancient languages, dingbats, mathematical symbols, and tons of punctuation. So if you have 16-bit faux strings, it’s very easy to think you have all of Unicode automatically handled and then be sorely mistaken. Thankfully, the increasing availability and popularity of emoji, which are mostly not in the BMP (but see below), makes astral plane support a more practical matter.

This probably all dates back to the original design of Unicode, which assumed that we’d never possibly need any more than 65,536 different characters and promised that two bytes would be enough for everyone. Oops.

(This is the same reason that Chinese hanzi and Japanese kanji are merged into a single set of codepoints: they’re both huge alphabets and it was the only way to fit them both into two bytes. This is called Han unification, and I have seen it end friendships, so I prefer not to discuss it further.)

One more trivium: MySQL has a utf8 encoding, and it’s generally regarded as best practice to use that for all your text columns so you can store Unicode. But, oops, MySQL arbitrarily limits it to three bytes per character, which isn’t enough to encode most astral plane characters! What a great technical decision and not at all yet another thorn in the unusable sinkhole that is MySQL. Version 5.5 introduced a utf8mb4 encoding that fixes this, so have fun ALTERing some multi-gigabyte tables in production.

There's no such thing as emoji

I exaggerate slightly.

The word “emoji” is generally used to mean “any character that shows as a colored picture on my screen”, much like the word “Unicode” is generally used to mean “any character not on my US QWERTY keyboard”. So what characters qualify as emoji?

There’s actually no Unicode block called “emoji”. The set of smiley faces is in a block called Emoticons, and most of the rest are in Miscellaneous Symbols and Pictographs and Transport and Map Symbols.

The Unicode Consortium has a technical report about emoji, which should be an immediate hint that this is not a trivial matter. In fact the report defines two levels of emoji, and look at how arbitrary these definitions are:

emoji character — A character that is recommended for use as emoji.

level 1 emoji character — An emoji character that is among those most commonly supported as emoji by vendors at present.

level 2 emoji character — An emoji character that is not a level 1 emoji character.

So emoji are defined somewhat arbitrarily, and even based on what’s treated as an emoji in the wild.

It’s tempting to just say that those few astral plane blocks are emoji, but you might be surprised at what else qualifies sometimes. There’s also a data table listing emoji levels, and it classifies as emoji a good handful of arrows and dingbats and punctuation, even though they’ve been in Unicode for many years. 🃏 U+1F0CF PLAYING CARD BLACK JOKER is a level 1 emoji, but nothing else in the entire Playing Cards block qualifies. Similarly, 🀄 U+1F004 MAHJONG TILE RED DRAGON is the only representative of Mahjong Tiles, and Domino Tiles aren’t represented at all.

I stress, also, that a colored graphic is not the only way emoji (however you define them) may be rendered. Here’s a screenshot of part of that table on my desktop:

Screenshot of emoji rendered as simple outlines

That font is Symbola, which only has monochrome vector glyphs. So they’re no different than any other character.

I’ve been seeing an increasing trend lately of treating emoji as somehow completely unique. The IM programs WhatsApp and Telegram both use Apple’s emoji font on every platform, and I’ve seen even technically-inclined people passionately argue that this is a good state of affairs, because it means both parties will see exactly the same pixels. Wouldn’t want to confuse anyone by having them see a slightly different image of a steak! (You’d think that’s what sending images is for, but what do I know.)

This is somewhat troubling to me. The entire point of having these symbols exist in Unicode is so they can be transferred between different systems and treated just like any other text, because now they’re just text. They aren’t special in any way (besides being in an astral plane, I suppose), and there’s no reason you couldn’t construct an emoji font that displayed regular English characters as graphics. Hell, if you’re using Firefox, here’s a demo of SVG embedded in an OpenType font that displays the letter “o” as an animated soccer ball. '''

In [21]:
print(len(bigger_text))

22788


In [22]:
# prompt: Read through the code in the above cells and replicate the same for this bigger text. i.e convert the characters in the string to UTF-8 tokens, find the stats using get_stats API, merge them to make the vocab size equal to 276 and print compression ratio

# Convert the characters in the bigger_text string to UTF-8 tokens.
bigger_tokens = bigger_text.encode('utf-8')
bigger_tokens = list(map(int,bigger_tokens))



# Merge the stats to make the vocab size equal to 276.
vocab_size = 276 # desired vocab_size
num_merges = vocab_size-256
ids = list(bigger_tokens) # copy of tokens
merges = {}
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats,key=stats.get)
  idx = 256+i
  ids = merge(ids,pair,idx)
  print(f'merging {pair} into {idx}')
  merges[pair] = idx

# Print the compression ratio.
compression_ratio = len(bigger_tokens) / len(ids)
print(f"Compression ratio: {compression_ratio}")


merging (101, 32) into 256
merging (32, 116) into 257
merging (32, 97) into 258
merging (105, 110) into 259
merging (101, 114) into 260
merging (116, 32) into 261
merging (115, 32) into 262
merging (257, 104) into 263
merging (226, 128) into 264
merging (111, 110) into 265
merging (104, 97) into 266
merging (101, 110) into 267
merging (121, 32) into 268
merging (44, 32) into 269
merging (100, 32) into 270
merging (111, 114) into 271
merging (97, 108) into 272
merging (259, 103) into 273
merging (97, 99) into 274
merging (99, 111) into 275
Compression ratio: 1.2574814973720905


In [23]:
print(len(bigger_tokens), len(ids))

23447 18646


the tokenization process is a completely independent one which is not related to LLMs. LLMs is only going to see the tokens as integers and no text is involved. Tokenization has its own training, processes etc.

Tokenization involves two processes : Encoding - conversion from texts to integers and decoding : conversion from integers back to python string

In [24]:
vocab = {idx : bytes([idx]) for idx in range(256)}
for (p0,p1), idx in merges.items():
  vocab[idx] = vocab[p0] + vocab[p1] # byte concatenation


In [25]:
#decoding
def decode(ids):
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode('utf-8',errors="replace")
  return text



In [26]:
decode([98])

'b'

Encoding string to tokens

In [35]:
def encode(text):
  tokens = list(text.encode('utf-8'))
  #Merge the tokens [byte pair encoding]
  while len(tokens)>=2:
    stats= get_stats(tokens)
    pair = min(stats, key = lambda p : merges.get(p,float("inf")))
    if pair not in merges:
      break # no more merging to do
    idx = merges[pair]
    tokens = merge(tokens,pair,idx)
  return tokens

In [36]:
print(encode("hello world!"))

[104, 101, 108, 108, 111, 32, 119, 271, 108, 100, 33]


In [38]:
print(encode('>'))

[62]


Some test cases

In [39]:
print(decode(encode("hello world")))

hello world


This is a simple tokenizer - large LLMs like GPT, Llama use a complicated tokenizer.

GPT tokenizer first splits the entire text into chunks of sub texts, because putting the entire text through encode + decode naively is sub-optimal, it loses some context and semantic understanding.
Instead using RegEx, it is split based on some rules and on these chunks of text, encode + decode processes are done to move back and forth between raw strings and integers used in transformers

In [42]:
import regex as re
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

some common apostrophes, 've are all taken as separate substrings for tokenization

let us experiment with openai tokenizer library - tiktoken

In [48]:
!pip install tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.7 MB/s eta 0:00:00


In [49]:
import tiktoken

In [53]:
enc_gpt2 = tiktoken.get_encoding('gpt2')
print(enc_gpt2.encode("   hello world!!!!"))
enc_gpt4 = tiktoken.get_encoding('cl100k_base')
print(enc_gpt4.encode("   hello world!!!"))

[220, 220, 23748, 995, 13896]
[256, 24748, 1917, 12340]


we see that in gpt2 , spaces are unmerged and tokens are allocated for each of the spaces, in gpt4 the spaces are merged.

in gpt2, the length of the vocab is 50257 tokens


In [54]:
!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json

--2024-02-25 11:43:23--  https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 20.60.179.33
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|20.60.179.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [application/octet-stream]
Saving to: ‘vocab.bpe’

vocab.bpe           100%[===================>] 445.62K  1.47MB/s    in 0.3s    

2024-02-25 11:43:23 (1.47 MB/s) - ‘vocab.bpe’ saved [456318/456318]

--2024-02-25 11:43:23--  https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 20.60.179.33
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|20.60.179.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [application/json]
Saving to: 

In [55]:
import os,json

with open('encoder.json','r') as f:
    encoder = json.load(f)

with open('vocab.bpe','r', encoding='utf-8') as f:
  bpe_data = f.read()

bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]


In [56]:
len(encoder) # 50257

50257

256 raw byte tokens and then there are 50000 merges. the last character is EOD [a special token]. kind of like reset after this special character is seen [a delimiter]

In [60]:
encoder['.']

13

more info on special tokens can be obtained by going throught he tiktoken library written in Rust

There are some other special tokens like End of Prompt, FiM_Prefix -> Fill in the middle, fill in the middle - suffix

TODO : Implement your own GPT 4 tokenizer based on the guidelines in minbpe repo and taking that as reference. Train the vocab on a large dataset having a lot of diverse character sets.

Another common tokenizer : Sentencepiece - can do both training and inference of tokenization , it is used by LLama and Mistral series.


the sentencepiece method applies bpe on the unicode points direclty instead of encoding the text using utf-8 bytes and then applying bpe.
It only falls back to byte level when there is a rare character or there is another hyperparamter called the character_coverage which is triggered when a before seen character appears.

In [61]:
#Example
import sentencepiece as spm

In [62]:
# write a toy.txt file with some random text
with open("toy.txt", "w", encoding="utf-8") as f:
  f.write("SentencePiece is an unsupervised text tokenizer and detokenizer mainly for Neural Network-based text generation systems where the vocabulary size is predetermined prior to the neural model training. SentencePiece implements subword units (e.g., byte-pair-encoding (BPE) [Sennrich et al.]) and unigram language model [Kudo.]) with the extension of direct training from raw sentences. SentencePiece allows us to make a purely end-to-end system that does not depend on language-specific pre/postprocessing.")

In [63]:
# train a sentencepiece model on it
# the settings here are (best effort) those used for training Llama 2
import os

options = dict(
  # input spec
  input="toy.txt",
  input_format="text",
  # output spec
  model_prefix="tok400", # output filename prefix
  # algorithm spec
  # BPE alg
  model_type="bpe",
  vocab_size=400,
  # normalization
  normalization_rule_name="identity", # ew, turn off normalization
  remove_extra_whitespaces=False,
  input_sentence_size=200000000, # max number of training sentences
  max_sentence_length=4192, # max number of bytes per sentence
  seed_sentencepiece_size=1000000,
  shuffle_input_sentence=True,
  # rare word treatment
  character_coverage=0.99995,
  byte_fallback=True,
  # merge rules
  split_digits=True,
  split_by_unicode_script=True,
  split_by_whitespace=True,
  split_by_number=True,
  max_sentencepiece_length=16,
  add_dummy_prefix=True,
  allow_whitespace_only_pieces=True,
  # special tokens
  unk_id=0, # the UNK token MUST exist
  bos_id=1, # the others are optional, set to -1 to turn off
  eos_id=2,
  pad_id=-1,
  # systems
  num_threads=os.cpu_count(), # use ~all system resources
)

spm.SentencePieceTrainer.train(**options)

Not all arguments here are useful, the sentence piece tokenizer has a historical context - it sees the raw stream of characters as sentences.hence the max_bytes per sentence, num_sentences etc. But according to Karpathy, it is better to keep the raw buffer as raw buffer itself -> just a stream of characters which are to be converted to tokens.

In [64]:
sp = spm.SentencePieceProcessor()
sp.load('tok400.model')
vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]

In [65]:
vocab

[['<unk>', 0],
 ['<s>', 1],
 ['</s>', 2],
 ['<0x00>', 3],
 ['<0x01>', 4],
 ['<0x02>', 5],
 ['<0x03>', 6],
 ['<0x04>', 7],
 ['<0x05>', 8],
 ['<0x06>', 9],
 ['<0x07>', 10],
 ['<0x08>', 11],
 ['<0x09>', 12],
 ['<0x0A>', 13],
 ['<0x0B>', 14],
 ['<0x0C>', 15],
 ['<0x0D>', 16],
 ['<0x0E>', 17],
 ['<0x0F>', 18],
 ['<0x10>', 19],
 ['<0x11>', 20],
 ['<0x12>', 21],
 ['<0x13>', 22],
 ['<0x14>', 23],
 ['<0x15>', 24],
 ['<0x16>', 25],
 ['<0x17>', 26],
 ['<0x18>', 27],
 ['<0x19>', 28],
 ['<0x1A>', 29],
 ['<0x1B>', 30],
 ['<0x1C>', 31],
 ['<0x1D>', 32],
 ['<0x1E>', 33],
 ['<0x1F>', 34],
 ['<0x20>', 35],
 ['<0x21>', 36],
 ['<0x22>', 37],
 ['<0x23>', 38],
 ['<0x24>', 39],
 ['<0x25>', 40],
 ['<0x26>', 41],
 ['<0x27>', 42],
 ['<0x28>', 43],
 ['<0x29>', 44],
 ['<0x2A>', 45],
 ['<0x2B>', 46],
 ['<0x2C>', 47],
 ['<0x2D>', 48],
 ['<0x2E>', 49],
 ['<0x2F>', 50],
 ['<0x30>', 51],
 ['<0x31>', 52],
 ['<0x32>', 53],
 ['<0x33>', 54],
 ['<0x34>', 55],
 ['<0x35>', 56],
 ['<0x36>', 57],
 ['<0x37>', 58],
 ['<0x38>', 5

as seen above, since byte_format was given as true,we have the special tokens first , then the bytecode tokens, then the unicode tokens encoded directly with merges.

In [66]:
ids = sp.encode("hello 안녕하세요")

In [67]:
print(ids)

[362, 378, 361, 372, 358, 362, 239, 152, 139, 238, 136, 152, 240, 152, 155, 239, 135, 187, 239, 157, 151]


the korean tokens are new and not encoutered in training set. so, they should have been unk tokens. but we have activated utf-8 bytecode tokens as well. so, the these korean letters have corresponding bytecode patterns which are part of vocab and those are printed

In [68]:
print([sp.id_to_piece(idx) for idx in ids])

['▁', 'h', 'e', 'l', 'lo', '▁', '<0xEC>', '<0x95>', '<0x88>', '<0xEB>', '<0x85>', '<0x95>', '<0xED>', '<0x95>', '<0x98>', '<0xEC>', '<0x84>', '<0xB8>', '<0xEC>', '<0x9A>', '<0x94>']


if we use byte_fallback - False, the korean lettes would be giveb the unknown token and this will limit the capabilities of the model.

In [81]:
ptions = dict(
  # input spec
  input="toy.txt",
  input_format="text",
  # output spec
  model_prefix="tok400", # output filename prefix
  # algorithm spec
  # BPE alg
  model_type="bpe",
  vocab_size=400,
  # normalization
  normalization_rule_name="identity", # ew, turn off normalization
  remove_extra_whitespaces=False,
  input_sentence_size=200000000, # max number of training sentences
  max_sentence_length=4192, # max number of bytes per sentence
  seed_sentencepiece_size=1000000,
  shuffle_input_sentence=True,
  # rare word treatment
  character_coverage=0.99995,
  byte_fallback=False, # to demonstrate the problems this would cause in korean case
  # merge rules
  split_digits=True,
  split_by_unicode_script=True,
  split_by_whitespace=True,
  split_by_number=True,
  max_sentencepiece_length=16,
  add_dummy_prefix=True,
  allow_whitespace_only_pieces=True,
  # special tokens
  unk_id=0, # the UNK token MUST exist
  bos_id=1, # the others are optional, set to -1 to turn off
  eos_id=2,
  pad_id=-1,
  # systems
  num_threads=os.cpu_count(), # use ~all system resources
)

spm.SentencePieceTrainer.train(**options)

In [84]:
sp = spm.SentencePieceProcessor()
sp.load('tok400.model')
vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]

In [85]:
vocab

[['<unk>', 0],
 ['<s>', 1],
 ['</s>', 2],
 ['<0x00>', 3],
 ['<0x01>', 4],
 ['<0x02>', 5],
 ['<0x03>', 6],
 ['<0x04>', 7],
 ['<0x05>', 8],
 ['<0x06>', 9],
 ['<0x07>', 10],
 ['<0x08>', 11],
 ['<0x09>', 12],
 ['<0x0A>', 13],
 ['<0x0B>', 14],
 ['<0x0C>', 15],
 ['<0x0D>', 16],
 ['<0x0E>', 17],
 ['<0x0F>', 18],
 ['<0x10>', 19],
 ['<0x11>', 20],
 ['<0x12>', 21],
 ['<0x13>', 22],
 ['<0x14>', 23],
 ['<0x15>', 24],
 ['<0x16>', 25],
 ['<0x17>', 26],
 ['<0x18>', 27],
 ['<0x19>', 28],
 ['<0x1A>', 29],
 ['<0x1B>', 30],
 ['<0x1C>', 31],
 ['<0x1D>', 32],
 ['<0x1E>', 33],
 ['<0x1F>', 34],
 ['<0x20>', 35],
 ['<0x21>', 36],
 ['<0x22>', 37],
 ['<0x23>', 38],
 ['<0x24>', 39],
 ['<0x25>', 40],
 ['<0x26>', 41],
 ['<0x27>', 42],
 ['<0x28>', 43],
 ['<0x29>', 44],
 ['<0x2A>', 45],
 ['<0x2B>', 46],
 ['<0x2C>', 47],
 ['<0x2D>', 48],
 ['<0x2E>', 49],
 ['<0x2F>', 50],
 ['<0x30>', 51],
 ['<0x31>', 52],
 ['<0x32>', 53],
 ['<0x33>', 54],
 ['<0x34>', 55],
 ['<0x35>', 56],
 ['<0x36>', 57],
 ['<0x37>', 58],
 ['<0x38>', 5

In [75]:
ids = sp.encode("hello 안녕하세요")

In [76]:
ids

[362,
 378,
 361,
 372,
 358,
 362,
 239,
 152,
 139,
 238,
 136,
 152,
 240,
 152,
 155,
 239,
 135,
 187,
 239,
 157,
 151]

a _ before the start of the sentence i.e before hello world is because of setting dummy_prefix = True

what should be vocab_size ?
in GPT code, the vocab_size is used in two places . the place where the embedding table is constructed and the final place where there is alinear layer which predicts the probabilitites for all vocab sizes.
with infinite or very large vocab size, the computations associated with final layer increase, the embedding table becoems bigger and num of paramters also increase.

To change the vocab size say to increase the number of tokens, the base model is frozen and the only the layers associated with vocab are changed. a small model surgery is enough.

There is also another optimization rather compression which is possible with tokenization - we take a chat language model like chatGPT and it is usually finetuend with prompt engineering, so instead of that, the base model is taken and a few more tokens are added and the other layers are frozen. the model is distilled to improve it performance without the need of prompts.

There are many problems associated with tokenization : in python due to spaces considered as separate tokens, it was not able to attend to many tokens. so,it was fixed in gpt4.
And non-English languages are not modelled properly.
not enough data to train and tokenization of non-English languages are expensive ,more tokens for semantically similar words.
potential attack from user end having knowledge of special tokens. |<end_of_text>| if passed , the processing abrupty stop for chatgpt.
Integer numbers are all tokenized with bizzare merges. instead of taking one 4 digit number as single token, it is split as 3-1 , 2-2, 1-3

Certain strings like SolidGoldMagiKarp which disturb the LLMs and make them go bizzare. they interpret them differently and produce unexpected outputs


how this happens is that, solidmagikarp is a reddit user account who is very active. and vocab was created with this dataset. so, there was a dedicated token for this user which was then never encountered during training. the result was that the embedding row vector for this token was untrained and was junk.
When a user tries to infer with this token, it junk passes forward into the transformer block and it gives rise to weird outputs
